In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

### Saves local subgraphs from the larger traffic graph for hops 1...10

In [2]:
import os, sys
sys.path.append('..')
from glob import glob
from configs import *
from tqdm import tqdm_notebook as tqdm
# from tqdm.auto import tqdm
# tqdm.monitor_interval = 0
from utils import *
import numpy as np
from time import time
import json
import numpy as np
from time import time, strptime, mktime
import matplotlib.pyplot as plt
import cv2
from time import sleep

In [15]:
with open('../data/man.json') as fl:
    vs, adjlist = json.load(fl)

In [16]:
print(len(vs))
print(len(adjlist))

684
684


In [32]:
def find_hops(hops, vs, adj):
    graphs = []
    cvs, cadj = complete_graph(vs, adj)
    
    for ri, root in enumerate(cvs):
        # search h-hops for each root
        visited = {}
        queue = [root]
        depth = {}
        depth[root] = 0
        exceeded = False
        
        maxd = 0
        while len(queue):
            head = queue.pop(0)
            visited[head] = True
            if depth[head] > maxd:
                maxd = depth[head]
            
            if depth[head] < hops:
                has_children = False
                
                for child in cadj[head]:
                    if child not in visited:
                        has_children = True
                        
                        queue.append(child)
                        depth[child] = depth[head] + 1
                if not has_children:
                    exceeded = True
                    break
        
#         if not exceeded:
        if root == '403487-403488' and hops == 2:
            print(depth, exceeded, visited)
        if exceeded:
            continue
        if maxd == hops:
            # if reached n-hops
            verts = [root]
            for vert in visited.keys(): 
                if vert not in verts: verts.append(vert)
            graphs.append((
                subgraph(verts, vs, adj), 
                depth
            ))
    
    
    return graphs

from utils import *

# for hops in range(1, 3):
for hops in range(1, 11):
    with open('../data/man.json') as fl:
        vs, adjlist = json.load(fl)
#     h_graphs = find_hops(hops, vs, adjlist)
    h_graphs = find_hops_2way(hops, vs, adjlist)
    
    rvs = len(h_graphs) / len(vs)
    
    print(hops, '%.1f%%' % (rvs * 100))
    sleep(1)
    
    if not os.path.isdir('images/hop_%d' % hops):
        os.mkdir('images/hop_%d' % hops)
    
    for ((iv, ia), depth) in tqdm(h_graphs):
        dlabel = lambda ent: '%d:%s' % (depth[ent], ent[0] + ent[-2:])
        gobj = show_graph(iv, ia, vdesc=dlabel)
        imname = 'images/hop_%d/%s_%d' % (hops, iv[0], hops)
        gobj.render(imname)
        
#         if iv[0] == '403487-403488':
#             print(len(iv))
        with open('../data/graphs/%s_n%d.json' % (iv[0], hops), 'w') as fl:
            json.dump([
                iv,
                ia
            ], fl, indent=4)
            
        
#         if iv[0] == '400258-400259':
#             gobj.render('images/samples/sample_%d' % hops)
# 1 95.8%
# 2 92.0%
# 3 88.3%
# 4 85.2%
# 5 82.6%
# 6 80.1%
# 7 77.6%
# 8 75.1%
# 9 72.5%
# 10 70.2%

1 90.9%


100%|██████████| 622/622 [00:10<00:00, 57.54it/s]


2 83.6%


100%|██████████| 572/572 [00:10<00:00, 56.42it/s]


3 77.2%


...

100%|██████████| 357/357 [00:07<00:00, 45.66it/s]


9 48.4%


100%|██████████| 331/331 [00:07<00:00, 44.65it/s]


10 44.9%


100%|██████████| 307/307 [00:07<00:00, 42.69it/s]


In [6]:
# with open('../data/graphs/400950-400951_n5.json') as fl:
#     lvs, ladjs = json.load(fl)
# print(len(lvs))

# with open('../data/manh_stops.json', 'w') as fl:
#     json.dump(lvs, fl, indent=4)

15


In [33]:
all_segs = {}
for fname in glob('../data/graphs/*.json'):
    with open(fname) as fl:
        lvs, _ = json.load(fl)
    for vert in lvs:
        all_segs[vert] = True
        
print(len(all_segs))

# with open('../data/manh_stops.json', 'w') as fl:
#     json.dump(list(all_segs.keys()), fl, indent=4)

682


### Track a subsample for smaller testing

In [34]:
roots = {}
for fname in glob('../data/graphs/*.json'):
    root = fname.split('/')[-1][:-5].split('_n')[0]
    if root not in roots: roots[root] = []
    roots[root].append(fname)
#     with open(fname) as fl:
#         lvs, _ = json.load(fl)
#     for vert in lvs:
#         all_segs[vert] = True
# all_segs = list(all_segs.keys())
print(len(roots))

622


In [35]:
has_n10 = [seg for seg in roots.keys() if any(['_n10' in fname for fname in roots[seg]])]
len(has_n10)

307

In [36]:
fours = [seg for seg in has_n10 if seg[0] == '4']
len(fours)

304

In [37]:
from random import shuffle
inds = [ii for ii in range(len(fours))]
shuffle(inds)

In [38]:
with open('../data/subset.txt', 'w') as fl:
    fl.write('\n'.join([fours[ind] for ind in inds[:19]]))

In [39]:
[fours[ind] for ind in inds[:19]]

['404171-402838',
 '400860-400861',
 '401264-401265',
 '403159-403160',
 '401271-401272',
 '404029-403870',
 '400118-400119',
 '401036-401037',
 '401460-402653',
 '401459-401460',
 '402626-400808',
 '400461-400462',
 '401331-401333',
 '403160-403161',
 '403116-405394',
 '402805-405057',
 '400516-405179',
 '401182-401183',
 '402819-404947']